# Dogs vs. Cats dataset - Convnet

## What will be doing?

- Training an small model from scratch
- Doing feature extraction using pretrained layers
- Fine-Tune the pretrained model

## Downloading the data

Data is available [here](https://www.kaggle.com/c/dogs-vs-cats/data).

In [1]:
from pathlib import Path
import shutil

In [2]:
ORIGINAL_DATASET_DIR = Path('../datasets/dogs_cats')
DATA_DIR = Path('data/dogs_cat_dataset')
REGENERATE_DATA = False

In [3]:
def move_data(fnames, dst_base_dir):
    for fname in fnames:
        src = ORIGINAL_DATASET_DIR.joinpath(fname)
        dst = dst_base_dir.joinpath(fname)
        shutil.copy(src, dst)

def maybe_create_dir(dir_):
    if not dir_.exists():
        dir_.mkdir()
        
train_dir = DATA_DIR.joinpath('train'); maybe_create_dir(train_dir)
validation_dir = DATA_DIR.joinpath('validation'); maybe_create_dir(validation_dir)
test_dir = DATA_DIR.joinpath('test'); maybe_create_dir(test_dir)

train_cats_dir = train_dir.joinpath('cats'); maybe_create_dir(train_cats_dir)
train_dogs_dir = train_dir.joinpath('dogs'); maybe_create_dir(train_dogs_dir)

validation_cats_dir = validation_dir.joinpath('cats'); maybe_create_dir(validation_cats_dir)
validation_dogs_dir = validation_dir.joinpath('dogs'); maybe_create_dir(validation_dogs_dir)

test_cats_dir = test_dir.joinpath('cats'); maybe_create_dir(test_cats_dir)
test_dogs_dir = test_dir.joinpath('dogs'); maybe_create_dir(test_dogs_dir)

In [4]:
if REGENERATE_DATA:
    train_cat_names = ['cat.{}.jpg'.format(i) for i in range(1000)]
    val_cat_names = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    test_cat_names = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]

    move_data(train_cat_names, train_cats_dir)
    move_data(val_cat_names, validation_cats_dir)
    move_data(test_cat_names, test_cats_dir)

    train_dog_names = ['dog.{}.jpg'.format(i) for i in range(1000)]
    val_dog_names = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    test_dog_names = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]

    move_data(train_dog_names, train_dogs_dir)
    move_data(val_dog_names, validation_dogs_dir)
    move_data(test_dog_names, test_dogs_dir)

## Frame the problem

Balanced binary-classification

## Build a simple model

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPool2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [12]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## Data preparation steps

1. Read the pictures files
2. Decode the JPEG content to RGB grids of pixels
3. Convert these into floating-point tensors
4. Rescale the pixel values to the [0, 1] interval (as you know, neural networks prefer small values)

In [2]:
import tensorflow as tf

In [3]:
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

InvalidArgumentError: Cannot assign a device for operation MatMul: node MatMul (defined at <ipython-input-3-366263745427>:4) was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[node MatMul (defined at <ipython-input-3-366263745427>:4) ]]

Caused by op 'MatMul', defined at:
  File "c:\users\guill\anaconda3\envs\p36_tf\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\guill\anaconda3\envs\p36_tf\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\guill\anaconda3\envs\p36_tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\guill\anaconda3\envs\p36_tf\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\gui

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    str(train_dir),
    target_size=(150, 150),
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    str(validation_dir),
    target_size=(150, 150),
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
for data_batch, labels_batch in train_generator:
    print(data_batch.shape)
    print(labels_batch.shape)
    break

(32, 150, 150, 3)
(32,)


## Train the model

In [14]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=30)

Epoch 1/30
63/63 [==============================] - 62s 976ms/step - loss: 0.6928 - acc: 0.5155 - val_loss: 0.6775 - val_acc: 0.5340
Epoch 2/30
63/63 [==============================] - 61s 964ms/step - loss: 0.6597 - acc: 0.5875 - val_loss: 0.6395 - val_acc: 0.6340
Epoch 3/30
63/63 [==============================] - 60s 950ms/step - loss: 0.6147 - acc: 0.6650 - val_loss: 0.6146 - val_acc: 0.6430
Epoch 4/30
63/63 [==============================] - 59s 944ms/step - loss: 0.5813 - acc: 0.6950 - val_loss: 0.6230 - val_acc: 0.6570
Epoch 5/30
45/63 [====================>.........] - ETA: 14s - loss: 0.5535 - acc: 0.7278

KeyboardInterrupt: 

In [ ]:
model.save('cats_and_dogs_small_1.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_los']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()